## Step 3 - Climate Analysis and Exploration #

You are now ready to use Python and SQLAlchemy to do basic climate analysis and data exploration on your new weather station tables. All of the following analysis should be completed using SQLAlchemy ORM queries, Pandas, and Matplotlib.

Create a Jupyter Notebook file called climate_analysis.ipynb and use it to complete your climate analysis and data exporation.
Choose a start date and end date for your trip. Make sure that your vacation range is approximately 3-15 days total.
Use SQLAlchemy create_engine to connect to your sqlite database.
Use SQLAlchemy automap_base() to reflect your tables into classes and save a reference to those classes called Station and Measurement.

In [ ]:
#Dependencies
import datetime as dt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect, desc
from sqlalchemy.sql import label

from flask import Flask, jsonify

In [ ]:
engine = create_engine("sqlite:///Resources/hawaii.sqlite", echo=False)

In [ ]:
Base = automap_base()
Base.prepare(engine, reflect=True)
Measurement = Base.classes.measurements
Stations = Base.classes.stations
session = Session(bind = engine)

In [ ]:
inspector = inspect(engine)
inspector.get_table_names()

In [ ]:
columns = inspector.get_columns('measurements')
for c in columns:
    print(c['name'], c["type"])
print('__________________________________')    
columns = inspector.get_columns('stations')
for c in columns:
    print(c['name'], c["type"])

### Precipitation Analysis

Design a query to retrieve the last 12 months of precipitation data.
Select only the date and prcp values.
Load the query results into a Pandas DataFrame and set the index to the date column.
Plot the results using the DataFrame plot method.
Use Pandas to print the summary statistics for the precipitation data.


In [ ]:
results = session.query(Measurement.date, Measurement.prcp).filter((Measurement.date >= '2016-08-23') &
                                                              (Measurement.date <='2017-08-23'))

In [ ]:
data = {'date': [], 'prcp': []}

for row in results:
    data['date'].append(row.date)
    data['prcp'].append(row.prcp)

prcp_year = pd.DataFrame(data, columns = ['prcp','date'] , index =data['date'])
prcp_year['date']= pd.to_datetime(prcp_year['date'], format='%Y-%m-%d')

print(prcp_year.head(10000))
prcp_year.dtypes

In [ ]:
#prcp_year.plot(prcp_year['prcp'], prcp_year['date'])
prcp_year.plot('date','prcp')
plt.xlabel("Date")
plt.ylabel("Rain (Inches)")
plt.title("Precipitation Hawaii (2016-08-23 to 2017-08-23)")
plt.legend(["Precipitation"])
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
prcp_year.describe()

### Station Analysis

Design a query to calculate the total number of stations.
Design a query to find the most active stations.
List the stations and observation counts in descending order
Which station has the highest number of observations?

Design a query to retrieve the last 12 months of temperature observation data (tobs).

Filter by the station with the highest number of observations.
Plot the results as a histogram with bins=12.


In [ ]:
stations = session.query(Measurement).group_by(Measurement.station).count()
print(stations)

In [ ]:
most_obs = session.query(Measurement.station, func.count(Measurement.tobs)).group_by(Measurement.station).\
               order_by(func.count(Measurement.tobs).desc()).all()

most = most_obs[0][0]    
print("Name:",most,'=',most_obs[0][1],"weather observations.")
print('_______________')
for station, count in most_obs:
    print("Name:",station,"=",count,"obs.")


In [ ]:
temp = session.query(Measurement.station, Measurement.date, Measurement.tobs).\
    filter(Measurement.station == most).\
    filter((Measurement.date >= '2016-08-23')&(Measurement.date <='2017-08-23')).order_by(Measurement.date).all()

temp_df=pd.DataFrame(temp)
plt.hist(temp_df['tobs'],12)
plt.xlabel("Recorded Temp")
plt.ylabel("Number of Observations")
plt.title("Station Analysis (2016-08-23 to 2017-08-23) for Station " + most)
plt.show()

### Temperature Analysis

Write a function called calc_temps that will accept a start date and end date in the format %Y-%m-%d and return the minimum, average, and maximum temperatures for that range of dates.
Use the calc_temps function to calculate the min, avg, and max temperatures for your trip using the matching dates from the previous year (i.e. use "2017-01-01" if your trip start date was "2018-01-01")

Plot the min, avg, and max temperature from your previous query as a bar chart.


Use the average temperature as the bar height.
Use the peak-to-peak (tmax-tmin) value as the y error bar (yerr).

In [ ]:
def calc_temps(start, end):
    return session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)).\
        filter(Measurement.date >= start).filter(Measurement.date <= end).all()

temp_range = (calc_temps('2017-01-01', '2018-01-01'))
print(temp_range)

In [ ]:
arrive = dt.date(2017, 11, 1)
leave = dt.date(2017, 11, 17)
last_year = dt.timedelta(days=365)
temp_last_year = (calc_temps((arrive-last_year), (leave-last_year)))

print(temp_last_year)

In [ ]:
import seaborn 
errval = temp_last_year[0][2] - temp_last_year[0][0]
y = [temp_last_year[0][1]]
x = 0
fig, ax = plt.subplots()
ax.set_ylabel("Temperature", fontsize=14)
ax.set_title("Trip Average Temp", fontsize=18)
ax.bar(x, y, width=.1, color="red", yerr=errval)
ax.set_ylim(0, 100)
ax.set_xlim(-.1, .1)
ax.set_xbound(lower=-.1, upper=.1)
ax.tick_params(axis='x', which='both', bottom='off', top='off', labelbottom='off') 
plt.show()

# Step 4 - Climate App

Now that you have completed your initial analysis, design a Flask api based on the queries that you have just developed.
Use FLASK to create your routes.

Routes
/api/v1.0/precipitation

Query for the dates and temperature observations from the last year.
Convert the query results to a Dictionary using date as the key and tobs as the value.
Return the json representation of your dictionary.
/api/v1.0/stations

Return a json list of stations from the dataset.
/api/v1.0/tobs

Return a json list of Temperature Observations (tobs) for the previous year
/api/v1.0/<start> and /api/v1.0/<start>/<end>

Return a json list of the minimum temperature, the average temperature, and the max temperature for a given start or start-end range.
When given the start only, calculate TMIN, TAVG, and TMAX for all dates greater than and equal to the start date.
When given the start and the end date, calculate the TMIN, TAVG, and TMAX for dates between the start and end date inclusive.